##Libraries and data prep

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from transformers import RobertaTokenizerFast, RobertaForTokenClassification
from torch.utils.data import Dataset, DataLoader
import pdb
import torch
from torch import cuda
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import os

In [ ]:
train_df = pd.read_csv('/content/gdrive/MyDrive/feedback-prize-2021/train.csv')

In [ ]:
train_df.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [ ]:
from sklearn.utils import resample
# Get the majority class count
majority_count = train_df['discourse_type'].value_counts().max()

# Get all the unique classes
classes = train_df['discourse_type'].unique()

# List to store the upsampled dataframes
df_list = []

# Iterate over all classes and upsample minority classes
for class_type in classes:
    df_class = train_df[train_df.discourse_type == class_type]
    df_class_upsampled = resample(df_class,
                                  replace=True,
                                  n_samples=majority_count,
                                  random_state=42)
    df_list.append(df_class_upsampled)

# Combine majority class with upsampled minority classes
df_upsampled = pd.concat(df_list)
train_df=df_upsampled


In [ ]:
train_df.head(40)

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
116896,C8D3EB90EB29,1.621536e+12,0.0,551.0,Do you ever want to skip school and stay at ho...,Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
10503,FE62304F4B86,1.623254e+12,0.0,87.0,Have you ever realized how driving everywhere ...,Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
82468,48DF9E5125A3,1.621533e+12,0.0,241.0,Google has had cars that can drive independent...,Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
79558,3E5E0038C60A,1.620598e+12,0.0,193.0,Driverless cars are coming. Driverless cars wo...,Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
87645,FEF2E1612F55,1.615564e+12,15.0,191.0,What are the chances in favor of keeping the e...,Lead,Lead 1,2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
97523,6EC2740A88BB,1.614710e+12,0.0,407.0,"You get upearly on November 6, 2012 ready to v...",Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
5279,0AD7535C58C0,1.623081e+12,63.0,506.0,The national safety council estimates that 1.4...,Lead,Lead 1,12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 2...
62939,1EB4D8BFB732,1.614973e+12,0.0,146.0,Dear principal what are community service: is ...,Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
85255,F44A8B59CF6A,1.621531e+12,0.0,106.0,"Driverless cars are starting to be created, an...",Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16
132235,409D312BDC8D,1.618251e+12,98.0,210.0,"When you make a decision, you want to make the...",Lead,Lead 1,16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 3...


In [ ]:
test_names, test_texts = [], []
for f in tqdm(list(os.listdir('/content/gdrive/MyDrive/feedback-prize-2021/test'))):
    test_names.append(f.replace('.txt', ''))
    test_texts.append(open('/content/gdrive/MyDrive/feedback-prize-2021/test/' + f, 'r').read())
test_texts = pd.DataFrame({'id': test_names, 'text': test_texts})
# test_texts['text'] = test_texts['text'].apply(lambda x:x.split())
test_texts.head()

100%|██████████| 5/5 [00:02<00:00,  1.95it/s]


,id,text
0,18409261F5C2,80% of Americans believe seeking multiple opin...
1,D46BCB48440A,"When people ask for advice,they sometimes talk..."
2,0FB0700DAF44,"During a group project, have you ever asked a ..."
3,DF920E0A7337,Have you ever asked more than one person for h...
4,D72CB1C11673,Making choices in life can be very difficult. ...


In [ ]:
#train_text_df.to_csv("/content/gdrive/MyDrive/feedback-prize-2021/train_text_df.csv", index=False)
train_text_df = pd.read_csv("/content/gdrive/MyDrive/feedback-prize-2021/train_text_df.csv")
train_text_df.head()


,id,text
0,F00B4D036D97,"First and formost, it is my belief that we sho..."
1,EFDCE6A9E77B,The principal has made a decision saying all s...
2,EFE2D4E09A10,"Dear, Senator\n\nThe\n\nElectoral college has ..."
3,EFE800DEFA4B,I think that the use of this technology to rea...
4,EFDEF0E460D1,Not A Face\n\nThe Face on Mars is just a natur...


### Convert predictionstring and discourse_type as IOB labels for NER



In [ ]:
train_df.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
116896,C8D3EB90EB29,1.621536e+12,0.0,551.0,Do you ever want to skip school and stay at ho...,Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
10503,FE62304F4B86,1.623254e+12,0.0,87.0,Have you ever realized how driving everywhere ...,Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
82468,48DF9E5125A3,1.621533e+12,0.0,241.0,Google has had cars that can drive independent...,Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
79558,3E5E0038C60A,1.620598e+12,0.0,193.0,Driverless cars are coming. Driverless cars wo...,Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
87645,FEF2E1612F55,1.615564e+12,15.0,191.0,What are the chances in favor of keeping the e...,Lead,Lead 1,2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...


In [ ]:
# IOB representation
all_entities = []
for i in tqdm(train_text_df.iterrows()):
    total = i[1]['text'].split(' ').__len__()  #no of words in the text
    start = -1
    entities = []
    for j in train_df[train_df['id'] == i[1]['id']].iterrows():
        discourse = j[1]['discourse_type']
        list_ix = j[1]['predictionstring'].split(' ')
        ent = [f"I-{discourse}" for ix in list_ix]
        ent[0] = f"B-{discourse}"
        ds = int(list_ix[0])
        de = int(list_ix[-1])
        if start < ds-1:
            ent_add = ['O' for ix in range(int(ds-1-start))]
            ent = ent_add + ent

        entities.extend(ent)

        start = de
    if len(entities) < total:
        ent_add = ["O" for ix in range(total-len(entities))]
        entities += ent_add
    else:
        entities = entities[:total]

    all_entities.append(entities)


15594it [06:36, 39.36it/s]


In [ ]:
# all_entities

In [ ]:
train_text_df['entities'] = all_entities

In [ ]:

train_text_df.head() ## for each text mapped corresponding entities

,id,text,entities
0,F00B4D036D97,"First and formost, it is my belief that we sho...","[B-Position, I-Position, I-Position, I-Positio..."
1,EFDCE6A9E77B,The principal has made a decision saying all s...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
2,EFE2D4E09A10,"Dear, Senator\n\nThe\n\nElectoral college has ...","[O, O, O, B-Lead, I-Lead, I-Lead, I-Lead, I-Le..."
3,EFE800DEFA4B,I think that the use of this technology to rea...,"[B-Position, I-Position, I-Position, I-Positio..."
4,EFDEF0E460D1,Not A Face\n\nThe Face on Mars is just a natur...,"[O, O, O, B-Position, I-Position, I-Position, ..."


In [ ]:
# prompt: split train_text into 80 20  train and validation dataset

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train_text_df, test_size=0.2, random_state= 22)


In [ ]:
val_df.head()

,id,text,entities
11346,40B094686B62,"Dear President of the United States,\n\nI thin...","[O, O, O, O, O, O, B-Position, I-Position, I-P..."
13120,1C592378CBD1,"After going to school for ten months, students...","[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
7649,87B9974CE965,Hi my name is STUDENT_NAME and i would like to...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
5847,AB199403D486,"Is Venus a worthy pursuit despite the danger, ...","[B-Position, I-Position, I-Position, I-Positio..."
3375,C3A3146784DC,Dear TEACHER_NAME\n\nWhat is wrong with having...,"[O, O, B-Lead, I-Lead, I-Lead, I-Lead, I-Lead,..."


In [ ]:
config = {'model_name': 'roberta-base',
         'max_length': 512,
         'train_batch_size': 8,
         'valid_batch_size': 16,
         'epochs': 3,
         'learning_rate': 1e-05,
         'max_grad_norm': 10,
         'device': 'cuda' if cuda.is_available() else 'cpu'}

### Mapping of labels and ids

In [ ]:
output_labels = ['O', 'B-Lead', 'I-Lead', 'B-Position', 'I-Position', 'B-Claim', 'I-Claim', 'B-Counterclaim', 'I-Counterclaim',
          'B-Rebuttal', 'I-Rebuttal', 'B-Evidence', 'I-Evidence', 'B-Concluding Statement', 'I-Concluding Statement']

labels_to_ids = {v:k for k,v in enumerate(output_labels)}
ids_to_labels = {k:v for k,v in enumerate(output_labels)}

In [ ]:
ids_to_labels

{0: 'O',
 1: 'B-Lead',
 2: 'I-Lead',
 3: 'B-Position',
 4: 'I-Position',
 5: 'B-Claim',
 6: 'I-Claim',
 7: 'B-Counterclaim',
 8: 'I-Counterclaim',
 9: 'B-Rebuttal',
 10: 'I-Rebuttal',
 11: 'B-Evidence',
 12: 'I-Evidence',
 13: 'B-Concluding Statement',
 14: 'I-Concluding Statement'}

In [ ]:
labels_to_ids

{'O': 0,
 'B-Lead': 1,
 'I-Lead': 2,
 'B-Position': 3,
 'I-Position': 4,
 'B-Claim': 5,
 'I-Claim': 6,
 'B-Counterclaim': 7,
 'I-Counterclaim': 8,
 'B-Rebuttal': 9,
 'I-Rebuttal': 10,
 'B-Evidence': 11,
 'I-Evidence': 12,
 'B-Concluding Statement': 13,
 'I-Concluding Statement': 14}

### dataset prep


In [ ]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)  #loading data, tokenizer and mexlen
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels
        sentence = self.data.text[index]
        word_labels = self.data.entities[index]

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
#                              is_pretokenized=True,
#                                   is_split_into_words=True,   #encoding the texts to get token and attention
                             return_offsets_mapping=True,
                             padding='max_length',
                             truncation=True,
                             max_length=self.max_len)

        # step 3: create token labels only for first word pieces of each tokenized word
#         pdb.set_trace()
        labels = [labels_to_ids[label] for label in word_labels]

        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100  # create an empty array of -100 of length max_length
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):

            if mapping[0] != 0 and mapping[0] != encoding['offset_mapping'][idx-1][1]:   #Getting offset labels with the help of return_offsets_mapping
                try:
                    encoded_labels[idx] = labels[i]
                except:
                    pass
                i += 1
            else:
                if idx==1:
                    encoded_labels[idx] = labels[i]
                    i += 1
        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}  #converting the input tokens, attention mask, and offset mappings to tensors.
        item['labels'] = torch.as_tensor(encoded_labels)   #assigning corresponding label to each encoded token

        return item

  def __len__(self):
        return self.len

In [ ]:
# Replace DeBERTa tokenizer with RoBERTa tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained(config['model_name'])

# Replace DeBERTa model with RoBERTa model
model = RobertaForTokenClassification.from_pretrained(config['model_name'], num_labels=len(output_labels))

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#print(model.config)

### Creating train test split and putting data into data loders

In [ ]:
data = train_text_df[['text', 'entities']]
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, config['max_length'])
testing_set = dataset(test_dataset, tokenizer, config['max_length'])

FULL Dataset: (15594, 2)
TRAIN Dataset: (12475, 2)
TEST Dataset: (3119, 2)


In [ ]:
data.head()

,text,entities
0,"First and formost, it is my belief that we sho...","[B-Position, I-Position, I-Position, I-Positio..."
1,The principal has made a decision saying all s...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
2,"Dear, Senator\n\nThe\n\nElectoral college has ...","[O, O, O, B-Lead, I-Lead, I-Lead, I-Lead, I-Le..."
3,I think that the use of this technology to rea...,"[B-Position, I-Position, I-Position, I-Positio..."
4,Not A Face\n\nThe Face on Mars is just a natur...,"[O, O, O, B-Position, I-Position, I-Position, ..."


In [ ]:
train_dataset.head()

,text,entities
0,"In the article ""The Challenge of Exploring Ven...","[B-Position, I-Position, I-Position, I-Positio..."
1,Allowing students to take classes online would...,"[B-Position, I-Position, I-Position, I-Positio..."
2,Are cars necessary in today's society? Of cour...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
3,So you want to do a extracurricular activity? ...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
4,Would you enjoy your summer if you were requir...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."


In [ ]:
#train_dataset['entities'][0:1][0]

In [ ]:
train_params = {'batch_size': config['train_batch_size'],
                'shuffle': True,
                'num_workers': 1,
                'pin_memory':True
                }

test_params = {'batch_size': config['valid_batch_size'],
                'shuffle': True,
                'num_workers': 1,
                'pin_memory':True
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
test_texts_set = dataset(test_texts, tokenizer, config['max_length'])
test_texts_loader = DataLoader(test_texts_set, **test_params)

### Gutcheck to make sure all shapes are correct and model able to run without errors

In [ ]:
device = config['device']

In [ ]:
#Gut check with a sample text
model.to(device)
inputs = training_set[2]
#print(inputs)
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels,
               return_dict=False)
initial_loss = outputs[0]
initial_loss

tensor(2.4742, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=config['learning_rate'])

In [ ]:
# # DTrain test 80% - 20% validation split
# def train(epoch):
#     tr_loss, tr_accuracy = 0, 0
#     nb_tr_examples, nb_tr_steps = 0, 0
#     tr_preds, tr_labels = [], []
#     # put model in training mode
#     model.train()

#     for idx, batch in enumerate(training_loader):

#         ids = batch['input_ids'].to(device, dtype = torch.long)
#         mask = batch['attention_mask'].to(device, dtype = torch.long)
#         labels = batch['labels'].to(device, dtype = torch.long)

#         loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels,
#                                return_dict=False)
#         tr_loss += loss.item()

#         nb_tr_steps += 1
#         nb_tr_examples += labels.size(0)

#         if idx % 100==0:
#             loss_step = tr_loss/nb_tr_steps
#             print(f"Training loss per {idx} training steps: {loss_step}")

#         # compute training accuracy
#         flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
#         active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
#         flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

#         # only compute accuracy at active labels
#         active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
#         #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))

#         labels = torch.masked_select(flattened_targets, active_accuracy)
#         predictions = torch.masked_select(flattened_predictions, active_accuracy)

#         tr_labels.extend(labels)
#         tr_preds.extend(predictions)

#         tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
#         tr_accuracy += tmp_tr_accuracy

#         # gradient clipping
#         torch.nn.utils.clip_grad_norm_(
#             parameters=model.parameters(), max_norm=config['max_grad_norm']
#         )

#         # backward pass
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     epoch_loss = tr_loss / nb_tr_steps
#     tr_accuracy = tr_accuracy / nb_tr_steps
#     print(f"Training loss epoch: {epoch_loss}")
#     print(f"Training accuracy epoch: {tr_accuracy}")

In [ ]:
# Initialize lists to store training and validation loss
train_loss = []
val_loss = []

# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['input_ids'].to(device, dtype=torch.long)
        mask = batch['attention_mask'].to(device, dtype=torch.long)
        labels = batch['labels'].to(device, dtype=torch.long)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels, return_dict=False)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=config['max_grad_norm'])

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    train_loss.append(epoch_loss)
    print(f"Training loss epoch: {epoch_loss}")

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():

        for idx, batch in enumerate(testing_loader):

            ids = batch['input_ids'].to(device, dtype=torch.long)
            mask = batch['attention_mask'].to(device, dtype=torch.long)
            labels = batch['labels'].to(device, dtype=torch.long)

            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels, return_dict=False)

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)

        epoch_val_loss = eval_loss / nb_eval_steps
        val_loss.append(epoch_val_loss)
        print(f"Validation loss: {epoch_val_loss}")



In [ ]:

for epoch in range(4):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss epoch: 1.191731892258693
Validation loss: 1.0934586002276494
Training epoch: 2
Training loss epoch: 1.0386790271370838
Validation loss: 1.0289634777949406
Training epoch: 3


In [ ]:
train_loss, val_loss

In [ ]:
# Save the model and tokenizer
output_dir = '/content/gdrive/MyDrive/feedback-prize-2021/actual_roberta_model'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

In [ ]:
# # prompt: read model from this directory '/content/gdrive/MyDrive/feedback-prize-2021/actual_roberta_model'

# #from transformers import RobertaTokenizerFast, RobertaForTokenClassification
# model = RobertaForTokenClassification.from_pretrained('/content/gdrive/MyDrive/feedback-prize-2021/actual_roberta_model')
# tokenizer = RobertaTokenizerFast.from_pretrained('/content/gdrive/MyDrive/feedback-prize-2021/actual_roberta_model')

In [ ]:
import matplotlib.pyplot as plt
epochs = len(train_loss)
epochs_range = range(1, epochs + 1)

plt.plot(epochs_range, train_loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('RoBERTA Training vs. Validation Loss')
plt.legend()
plt.show()

In [ ]:
# torch.save(model, '/content/gdrive/MyDrive/feedback-prize-2021/roberta_model.pth')

### Inference function

In [ ]:
sentence = "@HuggingFace is a company based in New York, but is also has employees working in Paris"
model.eval()
def inference(sentence):
    inputs = tokenizer(sentence,
#                         is_split_into_words=True,
                        return_offsets_mapping=True,
                        padding='max_length',
                        truncation=True,
                        max_length=config['max_length'],
                        return_tensors="pt")
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)

    # forward pass
    outputs = model(ids, attention_mask=mask, return_dict=False)
    logits = outputs[0]


    active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    prediction = []
    out_str = []
    off_list = inputs["offset_mapping"].squeeze().tolist()
    for idx, mapping in enumerate(off_list):
#         print(mapping, token_pred[1], token_pred[0],"####")

#         only predictions on first word pieces are important
        if mapping[0] != 0 and mapping[0] != off_list[idx-1][1]:
#             print(mapping, token_pred[1], token_pred[0])
            prediction.append(wp_preds[idx][1])
            out_str.append(wp_preds[idx][0])
        else:
            if idx == 1:
                prediction.append(wp_preds[idx][1])
                out_str.append(wp_preds[idx][0])
            continue
    return prediction, out_str

In [ ]:
test_texts = val_df[['id','text']]
test_texts.head()


In [ ]:

# test_texts = train_text_df['text'].tolist()[:10]
y_pred = []

for i, t in tqdm(enumerate(test_texts['text'].tolist())):
    o,o_t = inference(t)
    y_pred.append(o)
    l = train_text_df['entities'][i]


In [ ]:
final_preds = []
import pdb
for i in tqdm(range(len(test_texts))):
#     pdb.set_trace()
    idx = test_texts.id.values[i]
#     pred = ['']*len(test_texts[i])

#     for j in range(len(y_pred[i])):
#         if words[i][j] != None:
#             pred[words[i][j]] = labels[y_pred[i][j]]

    pred = [x.replace('B-','').replace('I-','') for x in y_pred[i]]
#     print(pred)
    preds = []
    j = 0
    while j < len(pred):
        cls = pred[j]
#         pdb.set_trace()
        if cls == 'O':
            j += 1
        end = j + 1
        while end < len(pred) and pred[end] == cls:
            end += 1

        if cls != 'O' and cls != '' and end - j > 10:
            final_preds.append((idx, cls, ' '.join(map(str, list(range(j, end))))))

        j = end
print(final_preds[1])

In [ ]:
test_preds=pd.DataFrame(final_preds, columns=['id', 'discourse_type', 'predictionstring'])
test_preds.head(20)


In [ ]:
id_list=list(test_preds['id'].unique())

In [ ]:
#Merging all test prediction strings based on id and discourse type
test_preds['predictionstring_merged'] = test_preds.groupby(['id', 'discourse_type'])['predictionstring'].transform('sum')
test_merged = test_preds.drop_duplicates(subset=['id','discourse_type','predictionstring_merged'])[['id','discourse_type','predictionstring_merged']]
test_merged.head(20)

#Accuracy

In [ ]:
train_df = pd.read_csv('/content/gdrive/MyDrive/feedback-prize-2021/train.csv')
train_df=train_df[train_df['id'].isin(id_list)]


In [ ]:
#Merging all train prediction strings based on id and discourse type
train_df['predictionstring_merged'] = train_df.groupby(['id', 'discourse_type'])['predictionstring'].transform('sum')
train_merged = train_df.drop_duplicates(subset=['id','discourse_type','predictionstring_merged'])[['id','discourse_type','predictionstring_merged']]
train_merged.head(20)






In [ ]:
#Final df with actuals and predicted values

final_df = pd.merge(train_merged, test_merged, how='left', on=['id','discourse_type'])
final_df.head()


In [ ]:
!pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import fuzz

def sim(str1, str2):
    return fuzz.partial_ratio(str1, str2)

# Assuming you have already created 'final_df' DataFrame

# Function to determine 'cm' based on similarity and NaN values
def determine_cm(row):
    if pd.notna(row['predictionstring_merged_y']):
        similarity_score = sim(str(row['predictionstring_merged_x']), str(row['predictionstring_merged_y']))
        if similarity_score > 0.5:
            return 'True Positive'
        else:
            return 'False Positive'
    else:
        return 'False Negative'

# Apply the function to create 'cm' column
final_df['cm'] = final_df.apply(determine_cm, axis=1)
final_df.rename(columns={'predictionstring_merged_x': 'actual_words', 'predictionstring_merged_y': 'predicted_words'}, inplace=True)

final_df.head(10)




In [ ]:
# prompt: rename final_df predictionstring_merged_x to actual words, predictionstring_merged_y to predicted words

final_df.rename(columns={'predictionstring_merged_x': 'actual_words', 'predictionstring_merged_y': 'predicted_words'}, inplace=True)


In [ ]:

final_df['discourse_type'].unique()

In [ ]:
final_df_lead = final_df[final_df['discourse_type'] == 'Lead']
final_df.head(10)
final_df_claim = final_df[final_df['discourse_type'] == 'Claim']
final_df_Evidence = final_df[final_df['discourse_type'] == 'Evidence']
final_df_Position = final_df[final_df['discourse_type'] == 'Position']
final_df_Rebuttal = final_df[final_df['discourse_type'] == 'Rebuttal']
final_df_Concluding_Statement = final_df[final_df['discourse_type'] == 'Concluding Statement']
final_df_Counterclaim = final_df[final_df['discourse_type'] == 'Counterclaim']

In [ ]:
# prompt: in the final_df cm column count the number of TruePositives and FalsePositives based on those vlaues calculate f-1 score

tp = final_df.loc[final_df['cm'] == 'True Positive'].shape[0]
fp = final_df.loc[final_df['cm'] == 'False Positive'].shape[0]
fn = final_df.loc[final_df['cm'] == 'False Negative'].shape[0]
f1_score = 2*tp/(2*tp + fp + fn)
print(f1_score)



In [1]:
def calculate_f1_score(df):
    tp = df.loc[df['cm'] == 'True Positive'].shape[0]
    fp = df.loc[df['cm'] == 'False Positive'].shape[0]
    fn = df.loc[df['cm'] == 'False Negative'].shape[0]
    f1_score = 2 * tp / (2 * tp + fp + fn)
    return f1_score

# Assuming you have final_df_lead, final_df_claim, final_df_Evidence, final_df_Position,
# final_df_Rebuttal, and final_df_Concluding_Statement DataFrames

# Calculate F1 scores for each DataFrame
f1_score_lead = calculate_f1_score(final_df_lead)
f1_score_claim = calculate_f1_score(final_df_claim)
f1_score_evidence = calculate_f1_score(final_df_Evidence)
f1_score_position = calculate_f1_score(final_df_Position)
f1_score_rebuttal = calculate_f1_score(final_df_Rebuttal)
f1_score_concluding_statement = calculate_f1_score(final_df_Concluding_Statement)
f1_score_Counterclaim = calculate_f1_score(final_df_Counterclaim)
f1_score_overall = calculate_f1_score(final_df)
# Print or use the F1 scores as needed
print("F1 Score for Lead:", f1_score_lead)
print("F1 Score for Claim:", f1_score_claim)
print("F1 Score for Evidence:", f1_score_evidence)
print("F1 Score for Position:", f1_score_position)
print("F1 Score for Rebuttal:", f1_score_rebuttal)
print("F1 Score for Concluding_Statement:", f1_score_concluding_statement)
print("F1 Score for Counterclaim:", f1_score_Counterclaim)
print("F1 Score for Overall:", f1_score_overall)


F1 Score for Lead: 0.9322740106186
F1 Score for Claim: 0.8146201512478
F1 Score for Evidence: 0.6577223732102
F1 Score for Position: 0.7250217469312
F1 Score for Rebuttal: 0.5218960217231
F1 Score for Concluding_Statement: 0.75559103161
F1 Score for Counterclaim: 0.6122082146132
F1 Score for Overall: 0.7561892376102


#Visualize

In [ ]:
# actual = train_df[[id,discourse_type,predictionstring]]
# actual.head()
actual=pd.DataFrame(train_df, columns=['id', 'discourse_type', 'predictionstring'])
actual.head()


In [ ]:
preds=pd.DataFrame(final_preds, columns=['id', 'discourse_type', 'predictionstring'])
preds.head()

In [ ]:
# prompt: save unique ids of preds to a list

id_list = list(preds['id'].unique())


In [ ]:
# Create 'discourse_start' and 'discourse_end' columns by splitting the 'predictionstring_merged_y' and converting the values to integers
preds['discourse_start'] = preds['predictionstring'].apply(lambda x: int(x.split()[0]) if isinstance(x, str) else None)
preds['discourse_end'] = preds['predictionstring'].apply(lambda x: int(x.split()[-1]) if isinstance(x, str) else None)

actual['discourse_start'] = actual['predictionstring'].apply(lambda x: int(x.split()[0]) if isinstance(x, str) else None)
actual['discourse_end'] = actual['predictionstring'].apply(lambda x: int(x.split()[-1]) if isinstance(x, str) else None)

In [ ]:
import spacy
import random
from spacy import displacy

colors = {
    'Lead': '#8000ff',
    'Position': '#2b7ff6',
    'Evidence': '#2adddd',
    'Claim': '#80ffb4',
    'Concluding Statement': '#d4dd80',
    'Counterclaim': '#ff8042',
    'Rebuttal': '#ff0000'
}

nlp = spacy.load('en_core_web_sm')  # Load SpaCy model

def read_essay(id):
    with open(f"/content/gdrive/MyDrive/feedback-prize-2021/train/{id}.txt") as f:
        essay = f.read()
    return essay

def word_to_char_offsets(text):
    doc = nlp(text)
    char_offsets = [(token.idx, token.idx + len(token.text)) for token in doc]
    return char_offsets

def visualize(example):
    ents = []
    for i, row in preds[preds['id'] == example].iterrows():
        start_word_num = int(row['discourse_start']) if pd.notna(row['discourse_start']) else None
        end_word_num = int(row['discourse_end']) if pd.notna(row['discourse_end']) else None

        if start_word_num is not None and end_word_num is not None:
            # Get the character offsets for each word in the essay
            char_offsets = word_to_char_offsets(read_essay(example))

            # Get the start and end character positions of the entity
            start_char_pos = char_offsets[start_word_num][0]
            end_char_pos = char_offsets[end_word_num][1]

            if start_char_pos is not None and end_char_pos is not None:
                ents.append({
                    'start': start_char_pos,
                    'end': end_char_pos,
                    'label': row['discourse_type']
                })

    data = read_essay(example)

    doc2 = {
        "text": data,
        "ents": ents,
        "title": "PREDICTED " + example
    }

    options = {"ents": preds.discourse_type.unique().tolist(), "colors": colors}
    displacy.render(doc2, style="ent", options=options, manual=True, jupyter=True)

def visualize_actual(example):
    ents = []
    for i, row in actual[actual['id'] == example].iterrows():
        start_word_num = int(row['discourse_start']) if pd.notna(row['discourse_start']) else None
        end_word_num = int(row['discourse_end']) if pd.notna(row['discourse_end']) else None

        if start_word_num is not None and end_word_num is not None:
            # Get the character offsets for each word in the essay
            char_offsets = word_to_char_offsets(read_essay(example))

            # Get the start and end character positions of the entity
            start_char_pos = char_offsets[start_word_num][0]
            end_char_pos = char_offsets[end_word_num][1]

            if start_char_pos is not None and end_char_pos is not None:
                ents.append({
                    'start': start_char_pos,
                    'end': end_char_pos,
                    'label': row['discourse_type']
                })

    data = read_essay(example)

    doc2 = {
        "text": data,
        "ents": ents,
        "title": "ACTUAL " + example
    }

    options = {"ents": actual.discourse_type.unique().tolist(), "colors": colors}
    displacy.render(doc2, style="ent", options=options, manual=True, jupyter=True)
#visualize("40B094686B62")

def visualize_random_example():
    random_id = random.choice(id_list)
    visualize_actual(random_id)
    visualize(random_id)

# Call the function to visualize a random example
visualize_random_example()